In [180]:
test1 = """AAAA
BBCD
BBCC
EEEC"""

In [181]:
test2 = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""

for each item in grid

check if it's in a region already

if not work out all connected items




In [182]:
def in_bound(pos, grid):
    a,b = pos
    if a >= 0 and a < len(grid):
        if b >= 0 and b < len(grid[0]):
            return True
    return False

In [183]:
def connected(pos, grid, region = set(), perim = 0):

    crop = grid[pos[0]][pos[1]]

    region.add(pos)

    perim += 4 #have 4 boundaries for each new position until we remove these

    directions = [(0,1),(0,-1),(1,0),(-1,0)]

    for dir in directions:
        new_pos = (pos[0] + dir[0], pos[1] + dir[1])
        if in_bound(new_pos,grid):
            if new_pos not in region: # not already in region
                if grid[new_pos[0]][new_pos[1]] == crop: # adjacent is same type 
                    new_regions, perim_update = connected(new_pos, grid, region, 0)
                    region = region.union(new_regions)
                    perim = perim - 1 + perim_update

            else: # adjacent is in a region so reduce perimeter
                perim = perim - 1
                    
    return region, perim

In [185]:
def get_regions(inp):
    grid = inp.splitlines()

    in_region = set()
    distinct_regions = []

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            start_region = (i,j)
            region = set()

            if not start_region in in_region: # if starting position is not already in a region
                new_region, perim = connected(start_region, grid, region, 0)
                region = region.union(new_region)
                in_region = in_region.union(region)
                distinct_regions.append([region,perim])

    return distinct_regions

In [186]:
def p1(inp):
    ans = 0
    all_regions = get_regions(inp)
    for regions, perim in all_regions:
        ans = ans + (len(regions)*perim)

    return ans

In [187]:
p1(test2)

1930

In [188]:
p1(open('data12').read())

1456082

In [189]:
def connected(pos, grid, region=None, fences=None):
    if region is None:
        region = set()
    if fences is None:
        fences = {}

    crop = grid[pos[0]][pos[1]]

    region.add(pos)

    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]


    fences[pos] = set(directions)  # have 4 fences for each new position until we remove these

    #print(f'pos: {pos}')
    for dir in directions:
        new_pos = (pos[0] + dir[0], pos[1] + dir[1])
        if in_bound(new_pos, grid):
            if new_pos not in region:  # not already in region
                #print(f'adding {new_pos} to {region}')

                if grid[new_pos[0]][new_pos[1]] == crop:  # adjacent is the same type
                    new_regions, new_fences = connected(new_pos, grid, region, fences)
                    region.update(new_regions)  # Merge new regions into the current region

                    # Merge fences from the new recursive call
                    for key, value in new_fences.items():
                        if key in fences:
                            fences[key].update(value)
                        else:
                            fences[key] = value

                    # Remove the fence because the other side is within the region
                    if pos in fences and dir in fences[pos]:
                        fences[pos].remove(dir)

            else:
                #print(f'fences: {fences}')
                if pos in fences and dir in fences[pos]:  # Remove fence because the other side is within region
                    #print(f'removing {dir} from {pos}')
                    fences[pos].remove(dir)

    return region, fences


In [190]:
def get_regions(inp):
    grid = inp.splitlines()

    in_region = set()
    distinct_regions = []

    for i in range(len(grid)):
        for j in range(len(grid[0])):
            start_region = (i,j)
            region = set()

            if not start_region in in_region: # if starting position is not already in a region
                new_region, fences = connected(start_region, grid, region)
                region = region.union(new_region)
                in_region = in_region.union(region)
                distinct_regions.append(fences)

    return distinct_regions

In [191]:
distinct_regions = get_regions(test2)
distinct_regions = get_regions(open('data12').read())

This is painful.

hacked together solution to order the list of fences, and count new side if there's a discontinous jump

In [ ]:
def neighbour(a,b, colchange=True):
    if a==b:
        raise ValueError("same position compared")
    
    if colchange:
        if a[1] != b[1]:
            return False
    else:
        if a[0] != a[0]:
            return False

    if abs(a[0]-b[0]) + abs(a[1]-b[1]) == 1:
        return True
    return False

In [ ]:
def p2(inp):
    distinct_regions = get_regions(inp)
    directions = [(0,1),(0,-1),(1,0),(-1,0)]
    region_sides = []
    ans = 0
    for region in distinct_regions:
        sides = 0
        directions_master = []

        for dir in directions:
            dir_list = []
            for k, v in region.items():
                if dir in v:
                    dir_list.append(k)
            directions_master.append(dir_list)
        
        count = 1
        for dir_item in directions_master:
            prev = (-100000,-100000)
            if count < 3:
                sorted_dir_item = sorted(dir_item, key=lambda x: (x[1], x[0]))
            else:
                sorted_dir_item = sorted(dir_item, key=lambda x: (x[0], x[1]))

            for item in sorted_dir_item:
                if count > 2:
                    colchange = False
                else:
                    colchange = True
                if not neighbour(prev,item, colchange):
                    sides += 1
                prev = item
            count += 1
        region_sides.append(sides)
        ans = ans + (sides*len(region))

    return(ans)
 

In [179]:
p2(test1)

80

In [176]:
p2(open('data12').read())

872382